#🔹 INSTALL REQUIRED LIBRARIES

In [1]:
!pip install streamlit
!pip install groq
!pip install PyMuPDF
!pip install fpdf
!pip install langdetect
!pip install cloudflared

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 49.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=8eb4af8a6631a8e6fd7c6995084f62ff594221554b9c38f43fef411ff36045d2
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=b4f7ae25429f7555e2431e9a487825d91d2c5f22aa4324af64bb2e66d65f6409
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1

#🔹 SET API KEY SAFELY

In [2]:
import os
os.environ["GROQ_API_KEY"] = "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"

#🔷 STEP 3: PDF TEXT EXTRACTION (NLP INGESTION)

In [3]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_file):
    text = ""
    doc = fitz.open(stream=pdf_file.read(), filetype="pdf")
    for page in doc:
        text += page.get_text()
    return text

#🔷 STEP 4: NLP PREPROCESSING (English Only)

In [4]:
from langdetect import detect

def clean_text(text):
    text = text.replace("\n", " ")
    text = text.strip()
    return text

def ensure_english(text):
    try:
        if detect(text) != "en":
            return "Translate to simple English:\n" + text
    except:
        pass
    return text

#🔷 STEP 5: LLM ANALYSIS (CORE LOGIC)

In [5]:
from groq import Groq

client = Groq()

def analyze_bill(text):
    prompt = f"""
You are a public policy analyst.

Analyze this Indian Parliament Bill and return:

1. Sector
2. Objective (bullet points)
3. Detailed Summary
4. Short-term Impact
5. Medium-term Impact
6. Long-term Impact

Use simple English.

Bill Text:
{text}
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )

    return response.choices[0].message.content

#🔷 STEP 6: STREAMLIT UI

In [3]:
#command: install streamlit and ngrok
!pip install streamlit pyngrok fpdf pymupdf groq


In [10]:
%%writefile app.py
import streamlit as st
from groq import Groq
from fpdf import FPDF
import fitz
import os
import re

# ================= PAGE CONFIG =================
st.set_page_config(page_title="Parliament Bill Auditor", layout="wide")
st.title("🏛️ Parliament Bill Auditor")

# ================= GROQ CLIENT =================
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# ================= SESSION STATE =================
st.session_state.setdefault("analysis", "")
st.session_state.setdefault("bill_text", "")
st.session_state.setdefault("chat_history", [])

# ================= PDF EXTRACTION =================
def extract_text_from_pdf(pdf_file):
    doc = fitz.open(stream=pdf_file.read(), filetype="pdf")
    return "\n".join(page.get_text() for page in doc)

# ================= TEXT CHUNKING =================
def chunk_text(text, size=1500):
    words = text.split()
    chunks, chunk = [], []
    for w in words:
        chunk.append(w)
        if len(" ".join(chunk)) > size:
            chunks.append(" ".join(chunk))
            chunk = []
    if chunk:
        chunks.append(" ".join(chunk))
    return chunks

# ================= BILL ANALYSIS =================
def analyze_bill(text):
    chunks = chunk_text(text)[:5]
    combined = "\n".join(chunks)

    prompt = f"""
You are a public policy analyst.

Analyze the bill and STRICTLY follow this format:

### SECTOR
Mention sector & sub-sector

### OBJECTIVES
- bullet points

### SUMMARY
Detailed explanation in simple English.

### POSITIVE IMPACT
- bullet points

### RISKS & CHALLENGES
- bullet points

### SHORT-TERM IMPACT
- bullet points

### MEDIUM-TERM IMPACT
- bullet points

### LONG-TERM IMPACT
- bullet points

Bill Text:
{combined}
"""
    r = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return r.choices[0].message.content

# ================= CHAT AI (FIXED) =================
def ask_bill_question(bill_text, analysis_text, question):
    prompt = f"""
You are a public policy assistant.

You may answer using:
1. Original bill text
2. AI-generated analysis

If information is unavailable in BOTH, say:
"Not mentioned in the bill or analysis."

--- BILL TEXT ---
{bill_text[:4000]}

--- ANALYSIS ---
{analysis_text[:4000]}

Question:
{question}
"""
    r = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return r.choices[0].message.content

# ================= SECTION PARSER =================
def extract_sections(text):
    sections = {
        "sector": "",
        "objectives": "",
        "summary": "",
        "positive": "",
        "risks": "",
        "impact": ""
    }
    current = None
    for line in text.splitlines():
        l = line.lower()
        if "### sector" in l:
            current = "sector"
        elif "### objective" in l:
            current = "objectives"
        elif "### summary" in l:
            current = "summary"
        elif "positive impact" in l:
            current = "positive"
        elif "risk" in l:
            current = "risks"
        elif "impact" in l:
            current = "impact"
        elif current:
            sections[current] += line + "\n"
    return sections

# ================= PDF GENERATOR =================
def generate_pdf(text):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=10)
    safe = re.sub(r"[^\x00-\xFF]", "-", text)
    for line in safe.split("\n"):
        pdf.multi_cell(0, 8, line)
    return pdf.output(dest="S").encode("latin-1")

# ================= UI =================
uploaded_file = st.file_uploader("📄 Upload Parliament Bill (PDF only)", type=["pdf"])

if uploaded_file:
    st.success("PDF uploaded successfully")
    st.session_state.bill_text = extract_text_from_pdf(uploaded_file)

    if st.button("🟢 GENERATE ANALYSIS"):
        with st.spinner("Analyzing large bill safely..."):
            st.session_state.analysis = analyze_bill(st.session_state.bill_text)

if st.session_state.analysis:
    sections = extract_sections(st.session_state.analysis)

    tab1, tab2, tab3 = st.tabs(["📊 Sector", "📝 Summary", "⚖️ Impact"])

    with tab1:
        st.subheader("📊 Sector Classification")
        st.write(sections["sector"])

    with tab2:
        st.subheader("🎯 Objectives")
        st.write(sections["objectives"])
        st.subheader("📝 Detailed Summary")
        st.write(sections["summary"])
        pdf = generate_pdf(sections["objectives"] + "\n\n" + sections["summary"])
        st.download_button("📥 Download Summary PDF", pdf, "bill_summary.pdf")

    with tab3:
        st.subheader("✅ Positive Impact")
        st.write(sections["positive"])
        st.subheader("⚠️ Risks & Challenges")
        st.write(sections["risks"])
        st.subheader("⏱ Short / Medium / Long Term Impact")
        st.write(sections["impact"])

# ================= CHAT UI =================
if st.session_state.bill_text and st.session_state.analysis:
    st.divider()
    st.subheader("💬 Ask AI about this Bill")

    question = st.text_input(
        "Ask a question about the current bill",
        placeholder="e.g. Explain long-term impact"
    )

    if st.button("Ask AI"):
        if question.strip():
            with st.spinner("Thinking..."):
                answer = ask_bill_question(
                    st.session_state.bill_text,
                    st.session_state.analysis,
                    question
                )
            st.session_state.chat_history.append({"q": question, "a": answer})
        else:
            st.warning("Please enter a question")

    for chat in reversed(st.session_state.chat_history):
        st.markdown(f"**🧑 Question:** {chat['q']}")
        st.markdown(f"**🤖 Answer:** {chat['a']}")
        st.markdown("---")


Overwriting app.py


In [5]:
!pip install pyngrok
from pyngrok import ngrok

In [6]:
# 🧹 Step 1: Clean up old installations and config
!pip uninstall -y pyngrok
!rm -rf /root/.ngrok2
!pip install pyngrok --quiet

# 🧠 Step 2: Import and add your valid authtoken
from pyngrok import ngrok
!ngrok config add-authtoken "34yRMi4zjk8EouYfzRqonOioZtm_3sCnf2k5ZYEVTsKUA7uUk"

# 🚀 Step 3: Run Streamlit + open tunnel correctly
!streamlit run app.py &>/dev/null&
public_url = ngrok.connect(8501)
print(f"✅ Streamlit App Running at: {public_url.public_url}")


Found existing installation: pyngrok 7.5.0
Uninstalling pyngrok-7.5.0:
  Successfully uninstalled pyngrok-7.5.0
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ Streamlit App Running at: https://subpopular-predominant-everette.ngrok-free.dev


In [ ]:
from pyngrok import ngrok

try:
    # Get a list of all active tunnels
    tunnels = ngrok.get_tunnels()

    if tunnels:
        print("Active tunnels:")
        for tunnel in tunnels:
            print(f"- {tunnel.public_url}")
            # Stop each tunnel individually
            ngrok.disconnect(tunnel.public_url)
        print("Attempted to disconnect active ngrok tunnels.")
    else:
        print("No active ngrok tunnels found started by this session.")

except Exception as e:
    print(f"An error occurred while trying to list or disconnect tunnels: {e}")
    print("You may need to manually stop ngrok processes or check your ngrok dashboard.")